In [1]:
import pandas as pd
import os
import gff3
import glob

In [35]:
file = '/Users/andreas/Bacillus/Bioinformatics/Scripts_RNAseq/subtiwiki.gene.export.2024-05-25.tsv'
subti = pd.read_csv(file, sep = '\t', header = 0)


# file = '/Users/andreas/Bacillus/Bioinformatics/Scripts_RNAseq/GenBank.gff3'
# genbank = pd.read_csv(file, sep = '\t', names = gff3.header, comment = '#')
file = '/Users/andreas/Bacillus/Bioinformatics/RNAseq_jag_khpa_kre/X204SC22071444-Z01-F001/Analysis/Annotation_files/B_subtilis_GenBank_w_name_and_locus.gff3'
genbank = pd.read_csv(file, sep = '\t', header = 0, comment = '#')

### some genes are on the form geneX_geneY
### split these genes into two rows; geneX and geneY

input_folder = '/Users/andreas/Bacillus/Bioinformatics/RNAseq_jag_khpa_kre/X204SC22071444-Z01-F001/Analysis/DEG'
output_folder = '/Users/andreas/Bacillus/Bioinformatics/RNAseq_jag_khpa_kre/X204SC22071444-Z01-F001/Analysis/DEG_significant'

# input_folder = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_30thMay2024/DPW/z_scores/Top_CRAC_hits'
# ouput_folder = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_30thMay2024/DPW/z_scores/Top_CRAC_hits/BSU'

# input_folder = '/Users/andreas/Bacillus/Papers_for_thesis/kre'
# output_folder = '/Users/andreas/Bacillus/Papers_for_thesis/kre/prepared'


# for file in glob.glob(input_folder + '/*.txt'):
#     df = pd.read_csv(file, sep = '\t', header = 0)
#     gene_base = []
#     for gene in df['Gene'].tolist():
#         if gene.find('NP_') == 0 and gene.count('_') == 1:
#             gene_base.append(gene.split('.')[0] + '.' + gene.split('.')[1])
#         else:
#             gene_base.append(gene.split('.')[0])
#     df['Gene_base'] = gene_base
#     df = df.sort_values(['padj'], ascending = True)
#     df = df.drop_duplicates(['Gene_base'])
#     df = df[abs(df['log2FoldChange']) >= 1]
#     df.to_csv(file, sep = '\t', header = True, index = False)

for file in glob.glob(input_folder + '/*.txt'):

    df = pd.read_csv(file, sep = '\t', header = 0)
    gene_base = []
    for gene in df['Gene'].tolist():
        if gene.find('NP_') == 0 and gene.count('_') == 1:
            gene_base.append(gene.split('.')[0] + '.' + gene.split('.')[1])
        else:
            gene_base.append(gene.split('.')[0])
    df['Gene_base'] = gene_base
    df = df.sort_values(['padj'], ascending = True)
    df = df.drop_duplicates(['Gene_base'])

    print(f'{os.path.basename(file)}: {len(df[(abs(df["log2FoldChange"]) >= 1) & (df["padj"] <= 0.05)])}')
    print(f'{os.path.basename(file)}: {len(df[(abs(df["log2FoldChange"]) >= 0.5) & (df["padj"] <= 0.05)])}')
    # df = df[abs(df['log2FoldChange']) >= 1]
    # df.to_csv(file, sep = '\t', header = True, index = False)

    all = pd.read_csv(file, sep = '\t', header = 0)
    all = all[abs(all['log2FoldChange']) >= 1]

    ### replace 'BSU_' with 'BSU^' 
    ### This way we can seperate transcripts by _ followingly
    temp_gene_names = []
    for name in all['Gene_base'].tolist():
        temp_gene_names.append(name.replace('BSU_', 'BSU^').replace('NP_', 'NP^').replace('YP_', 'YP^').replace('Novel_transcript', 'Novel^transcript'))
    all['Gene_base'] = temp_gene_names

    _df = pd.DataFrame()

    # Loop through each row in the original DataFrame
    for index, row in all.iterrows():
        # Check if 'Gene_base' contains an underscore
        if '_' in row['Gene_base']:
            # Split the 'Gene_base' value into a list of genes
            gene_list = row['Gene_base'].split('_')
            # Create new rows for each gene in gene_list
            for gene in gene_list:
                new_row = pd.DataFrame({
                    'Gene_base': [gene],
                    'baseMean': [row['baseMean']],
                    'log2FoldChange': [row['log2FoldChange']],
                    'lfcSE': [row['lfcSE']],
                    'stat': [row['stat']],
                    'pvalue': [row['pvalue']],
                    'padj': [row['padj']]
                })
                _df = pd.concat([_df, new_row], ignore_index=True)
        ### replace '@' with ' ' in gene names
        elif '@' in row['Gene_base']:
            new_row = pd.DataFrame({
                    'Gene_base': [row['Gene_base'].replace('@', ' ')],
                    'baseMean': [row['baseMean']],
                    'log2FoldChange': [row['log2FoldChange']],
                    'lfcSE': [row['lfcSE']],
                    'stat': [row['stat']],
                    'pvalue': [row['pvalue']],
                    'padj': [row['padj']]
                })
            _df = pd.concat([_df, new_row], ignore_index=True)

        else:
            # If nothing else, keep the row as is
            _df = pd.concat([_df, all[all.index == index]], ignore_index=True)
    
    ### replace 'BSU_' with 'BSU^' 
    ### This way we can seperate transcripts by _ followingly
    revert_gene_names = []
    for name in _df['Gene_base'].tolist():
        revert_gene_names.append(name.replace('BSU^', 'BSU_').replace('NP^', 'NP_').replace('YP^', 'YP_').replace('Novel^transcript', 'Novel_transcript'))
    _df['Gene_base'] = revert_gene_names

    ### make list of DEG
    sig_genes = list(set(_df['Gene_base'][_df['padj'] <= 0.05].tolist()))

    ### convert gene names to BSU_numbers
    bsu_numbers = []
    # for gene in sig_genes:
    #     try:
    #         bsu_numbers.append(subti['locus'][subti['title'] == gene].iloc[0])
    #     except IndexError:
    #         x = 0
    #         for index, row in subti.iterrows():
    #             try:
    #                 if gene in row['synonyms']:
    #                     bsu_numbers.append(subti['locus'][subti.index == index].iloc[0])
    #                     x += 1
    #                     break
    #             ## for na values
    #             except TypeError:
    #                 pass
    #         if x == 0:
    #             print(f'Error: Couldnt convert {gene}')
    for gene in sig_genes:
        try:
            bsu_numbers.append(subti['locus'][subti['title'] == gene].iloc[0])
            continue
        except IndexError:
            x = 0
            for index, row in subti.iterrows():
                try:
                    if gene in row['synonyms']:
                        bsu_numbers.append(subti['locus'][subti.index == index].iloc[0])
                        x += 1
                        break
                ## for na values
                except TypeError:
                    pass
            if x == 0:
                try:
                    bsu_numbers.append(genbank['locusTag'][genbank['name'] == gene].iloc[0])
                    continue
                except IndexError:
                    print(f'Error: Couldnt convert {gene}')
                    bsu_numbers.append('no_locusTag')
    
    bsunumbers = [i.replace('_', '') for i in bsu_numbers]

    ### convert bsu_numbers to genbank names
    genbank_names = []
    for bsu in bsu_numbers:
        for index, att in enumerate(genbank['attributes'].tolist()):
            x = 0
            if bsu in att:
                att_list = att.split(';')
                for a in att_list:
                    if a.find('gene=') == 0:
                        genbank_names.append(a[5:])
                        x += 1
                        break
            if x == 1:
                break
    
    ### Write list to .txt file
    # Specify the file name
    file_name = f"{output_folder}/{os.path.basename(file)}_sig_genes.txt"

    # Open the file in write mode
    with open(file_name, "w") as filex:
        # Iterate through each item in the list
        for item in list(set(genbank_names)):
            # Write the item followed by a newline character
            filex.write(f"{item}\n")

    file_name = f"{output_folder}/{os.path.basename(file)}_sig_genes_BSU_Numbers.txt"

    # Open the file in write mode
    with open(file_name, "w") as filex:
        # Iterate through each item in the list
        for item in list(set(bsu_numbers)):
            # Write the item followed by a newline character
            filex.write(f"{item}\n")

    file_name = f"{output_folder}/{os.path.basename(file)}_sig_genes_BSUNumbers_no_underscore.txt"

    # Open the file in write mode
    with open(file_name, "w") as filex:
        # Iterate through each item in the list
        for item in list(set(bsunumbers)):
            # Write the item followed by a newline character
            filex.write(f"{item}\n")


khpA_DEG.txt: 23
khpA_DEG.txt: 24
kre_DEG.txt: 76
kre_DEG.txt: 231
Error: Couldnt convert Novel_transcript
Error: Couldnt convert YP_009514004
jag_DEG.txt: 7
jag_DEG.txt: 103


In [3]:
### convert gene names to BSU_numbers on whole of DEG file
DEG_folder = '/Users/andreas/Bacillus/Bioinformatics/RNAseq_jag_khpa_kre/X204SC22071444-Z01-F001/Analysis/DEG'

for file in glob.glob(DEG_folder + '/*.txt'):
    all = pd.read_csv(file, sep = '\t', header = 0)

    ### replace 'BSU_' with 'BSU^' 
    ### This way we can seperate transcripts by _ followingly
    temp_gene_names = []
    for name in all['Gene_base'].tolist():
        temp_gene_names.append(name.replace('BSU_', 'BSU^').replace('NP_', 'NP^').replace('YP_', 'YP^').replace('Novel_transcript', 'Novel^transcript'))
    all['Gene_base'] = temp_gene_names

    _df = pd.DataFrame()

    # Loop through each row in the original DataFrame
    for index, row in all.iterrows():
        # Check if 'Gene_base' contains an underscore
        if '_' in row['Gene_base']:
            # Split the 'Gene_base' value into a list of genes
            gene_list = row['Gene_base'].split('_')
            # Create new rows for each gene in gene_list
            for gene in gene_list:
                new_row = pd.DataFrame({
                    'Gene': [row['Gene']],
                    'Gene_base': [gene],
                    'baseMean': [row['baseMean']],
                    'log2FoldChange': [row['log2FoldChange']],
                    'lfcSE': [row['lfcSE']],
                    'stat': [row['stat']],
                    'pvalue': [row['pvalue']],
                    'padj': [row['padj']]
                })
                _df = pd.concat([_df, new_row], ignore_index=True)
        ### replace '@' with ' ' in gene names
        elif '@' in row['Gene_base']:
            new_row = pd.DataFrame({
                    'Gene': [row['Gene']],
                    'Gene_base': [row['Gene_base'].replace('@', ' ')],
                    'baseMean': [row['baseMean']],
                    'log2FoldChange': [row['log2FoldChange']],
                    'lfcSE': [row['lfcSE']],
                    'stat': [row['stat']],
                    'pvalue': [row['pvalue']],
                    'padj': [row['padj']]
                })
            _df = pd.concat([_df, new_row], ignore_index=True)

        else:
            # If nothing else, keep the row as is
            _df = pd.concat([_df, all[all.index == index]], ignore_index=True)
    
    ### replace 'BSU_' with 'BSU^' 
    ### This way we can seperate transcripts by _ followingly
    revert_gene_names = []
    for name in _df['Gene_base'].tolist():
        revert_gene_names.append(name.replace('BSU^', 'BSU_').replace('NP^', 'NP_').replace('YP^', 'YP_').replace('Novel^transcript', 'Novel_transcript'))
    _df['Gene_base'] = revert_gene_names


    bsu_numbers = []
    for gene in _df['Gene_base'].tolist():
        try:
            bsu_numbers.append(subti['locus'][subti['title'] == gene].iloc[0])
        except IndexError:
            x = 0
            for index, row in subti.iterrows():
                try:
                    if gene in row['synonyms']:
                        bsu_numbers.append(subti['locus'][subti.index == index].iloc[0])
                        x += 1
                        break
                ## for na values
                except TypeError:
                    pass
            if x == 0:
                print(f'Error: Couldnt convert {gene}')
                bsu_numbers.append(gene)
    
    _df['locus'] = bsu_numbers
    _df.to_csv(file, sep = '\t', header = True, index = False)

Error: Couldnt convert frxB
Error: Couldnt convert ypzK
Error: Couldnt convert clsC
Error: Couldnt convert feuV
Error: Couldnt convert pbtP
Error: Couldnt convert pbtO
Error: Couldnt convert tboH
Error: Couldnt convert pelA
Error: Couldnt convert pbtN
Error: Couldnt convert rluD
Error: Couldnt convert rtbD
Error: Couldnt convert spdL
Error: Couldnt convert rttN
Error: Couldnt convert pyaA
Error: Couldnt convert clsC
Error: Couldnt convert hisK
Error: Couldnt convert pxpI
Error: Couldnt convert spxA
Error: Couldnt convert fmnP
Error: Couldnt convert akrN
Error: Couldnt convert rgpH
Error: Couldnt convert hisK
Error: Couldnt convert msrC
Error: Couldnt convert bioK
Error: Couldnt convert rluD
Error: Couldnt convert bioFC
Error: Couldnt convert walL
Error: Couldnt convert spdL
Error: Couldnt convert walL
Error: Couldnt convert lysP
Error: Couldnt convert rbn
Error: Couldnt convert rttO
Error: Couldnt convert tboT
Error: Couldnt convert pxpR
Error: Couldnt convert fmnP
Error: Couldnt conve

In [103]:
_df

,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,sig,Gene_base,locus
0,ygxB.2,3247.827950,-1.096988,0.133340,-8.226980,1.919932e-16,4.980303e-13,1.0,ygxB,BSU_09390
1,ydaG.2,7111.139000,-0.717854,0.086768,-8.273288,1.303140e-16,4.980303e-13,1.0,ydaG,BSU_04220
2,ydbD.2,1382.085553,-0.907377,0.113316,-8.007472,1.170903e-15,2.277991e-12,1.0,ydbD,BSU_04430
3,yflT.2,5008.471201,-0.801242,0.100872,-7.943139,1.971271e-15,3.068086e-12,1.0,yflT,BSU_07550
4,ohrB.2,3527.509671,-0.721755,0.092329,-7.817180,5.401965e-15,7.006349e-12,1.0,ohrB,BSU_13160
...,...,...,...,...,...,...,...,...,...,...
10688,manP_manA.1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,manA,BSU_12020
10689,S666.1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,S666,new_1925540_1925623_c
10690,S678.1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,S678,new_1949513_1949584_c
10691,BSU_19915.1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,BSU_19915,BSU_19915


In [21]:
### prepare CRAC to GO analysis
file = '/Users/andreas/Bacillus/Bioinformatics/Scripts_RNAseq/subtiwiki.gene.export.2024-05-25.tsv'
subti = pd.read_csv(file, sep = '\t', header = 0)


# file = '/Users/andreas/Bacillus/Bioinformatics/Scripts_RNAseq/GenBank.gff3'
# genbank = pd.read_csv(file, sep = '\t', names = gff3.header, comment = '#')
file = '/Users/andreas/Bacillus/Bioinformatics/RNAseq_jag_khpa_kre/X204SC22071444-Z01-F001/Analysis/Annotation_files/B_subtilis_GenBank_w_name_and_locus.gff3'
genbank = pd.read_csv(file, sep = '\t', header = 0, comment = '#')

### some genes are on the form geneX_geneY
### split these genes into two rows; geneX and geneY

input_folder = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_30thMay2024/ratios'
output_folder = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_30thMay2024/ratios/prepared'

# input_folder = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_30thMay2024/DPW/z_scores/Top_CRAC_hits'
# ouput_folder = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_30thMay2024/DPW/z_scores/Top_CRAC_hits/BSU'


for file in glob.glob(input_folder + '/*.txt'):
    df = pd.read_csv(file, sep = '\t', header = 0)
    gene_base = []
    for gene in df['Gene'].tolist():
        if gene.find('NP_') == 0 and gene.count('_') == 1:
            gene_base.append(gene.split('.')[0] + '.' + gene.split('.')[1])
        else:
            gene_base.append(gene.split('.')[0])
    df['Gene_base'] = gene_base
    df = df.drop_duplicates(['Gene_base'])
    df.to_csv(file, sep = '\t', header = True, index = False)

for file in glob.glob(input_folder + '/*.txt'):
    all = pd.read_csv(file, sep = '\t', header = 0)

    ### replace 'BSU_' with 'BSU^' 
    ### This way we can seperate transcripts by _ followingly
    temp_gene_names = []
    for name in all['Gene_base'].tolist():
        temp_gene_names.append(name.replace('BSU_', 'BSU^').replace('NP_', 'NP^').replace('YP_', 'YP^').replace('Novel_transcript', 'Novel^transcript'))
    all['Gene_base'] = temp_gene_names

    _df = pd.DataFrame()

    # Loop through each row in the original DataFrame
    for index, row in all.iterrows():
        # Check if 'Gene_base' contains an underscore
        if '_' in row['Gene_base']:
            # Split the 'Gene_base' value into a list of genes
            gene_list = row['Gene_base'].split('_')
            # Create new rows for each gene in gene_list
            for gene in gene_list:
                new_row = pd.DataFrame({
                    'Gene_base': [gene]
                })
                _df = pd.concat([_df, new_row], ignore_index=True)
        ### replace '@' with ' ' in gene names
        elif '@' in row['Gene_base']:
            new_row = pd.DataFrame({
                    'Gene_base': [row['Gene_base'].replace('@', ' ')]
                })
            _df = pd.concat([_df, new_row], ignore_index=True)

        else:
            # If nothing else, keep the row as is
            all_temp = pd.DataFrame()
            all_temp['Gene_base'] = all['Gene_base'][all.index == index].tolist()
            _df = pd.concat([_df, all_temp], ignore_index=True)
    
    
    ### replace 'BSU_' with 'BSU^' 
    ### This way we can seperate transcripts by _ followingly
    revert_gene_names = []
    for name in _df['Gene_base'].tolist():
        revert_gene_names.append(name.replace('BSU^', 'BSU_').replace('NP^', 'NP_').replace('YP^', 'YP_').replace('Novel^transcript', 'Novel_transcript'))
    _df['Gene_base'] = revert_gene_names
    _df = _df.drop_duplicates(['Gene_base'])

    ### make list of DEG
    sig_genes = _df['Gene_base'].tolist()

    ### convert gene names to BSU_numbers
    bsu_numbers = []
    # for gene in sig_genes:
    #     try:
    #         bsu_numbers.append(subti['locus'][subti['title'] == gene].iloc[0])
    #     except IndexError:
    #         x = 0
    #         for index, row in subti.iterrows():
    #             try:
    #                 if gene in row['synonyms']:
    #                     bsu_numbers.append(subti['locus'][subti.index == index].iloc[0])
    #                     x += 1
    #                     break
    #             ## for na values
    #             except TypeError:
    #                 pass
    #         if x == 0:
    #             print(f'Error: Couldnt convert {gene}')
    for gene in sig_genes:
        try:
            bsu_numbers.append(subti['locus'][subti['title'] == gene].iloc[0])
            continue
        except IndexError:
            x = 0
            for index, row in subti.iterrows():
                try:
                    if gene in row['synonyms']:
                        bsu_numbers.append(subti['locus'][subti.index == index].iloc[0])
                        x += 1
                        break
                ## for na values
                except TypeError:
                    pass
            if x == 0:
                try:
                    bsu_numbers.append(genbank['locusTag'][genbank['name'] == gene].iloc[0])
                    continue
                except IndexError:
                    print(f'Error: Couldnt convert {gene}')
                    bsu_numbers.append('no_locusTag')
    
    bsunumbers = [i.replace('_', '') for i in bsu_numbers]

    ### convert bsu_numbers to genbank names
    genbank_names = []
    for bsu in bsu_numbers:
        for index, att in enumerate(genbank['attributes'].tolist()):
            x = 0
            if bsu in att:
                att_list = att.split(';')
                for a in att_list:
                    if a.find('gene=') == 0:
                        genbank_names.append(a[5:])
                        x += 1
                        break
            if x == 1:
                break
    
    ### Write list to .txt file
    # Specify the file name
    file_name = f"{output_folder}/{os.path.basename(file)}_sig_genes.txt"

    # Open the file in write mode
    with open(file_name, "w") as filex:
        # Iterate through each item in the list
        for item in genbank_names:
            # Write the item followed by a newline character
            filex.write(f"{item}\n")

    file_name = f"{output_folder}/{os.path.basename(file)}_sig_genes_BSU_Numbers.txt"

    # Open the file in write mode
    with open(file_name, "w") as filex:
        # Iterate through each item in the list
        for item in bsu_numbers:
            # Write the item followed by a newline character
            filex.write(f"{item}\n")

    file_name = f"{output_folder}/{os.path.basename(file)}_sig_genes_BSUNumbers_no_underscore.txt"

    # Open the file in write mode
    with open(file_name, "w") as filex:
        # Iterate through each item in the list
        for item in bsunumbers:
            # Write the item followed by a newline character
            filex.write(f"{item}\n")


Error: Couldnt convert YP_009514003
Error: Couldnt convert YP_009513941
Error: Couldnt convert YP_009514007
Error: Couldnt convert YP_009513997
Error: Couldnt convert YP_009513943
Error: Couldnt convert Novel_transcript
Error: Couldnt convert Novel_transcript
Error: Couldnt convert YP_009514013
Error: Couldnt convert YP_009513997
Error: Couldnt convert YP_009514003
Error: Couldnt convert YP_009514007
Error: Couldnt convert Novel_transcript
Error: Couldnt convert YP_009513989


In [30]:
### process microarray dataset from Kre novel feedback loop paper
file = '/Users/andreas/Bacillus/Papers_for_thesis/kre_microarray_novel_feedbackloop_paper.tsv'
df = pd.read_csv(file, sep = '\t', header = 0)

In [31]:
sig = df[(df['adj.P.Val'] <= 0.05) & (df['logFC'] < 0)]

In [32]:
bsu = sig['ID'].tolist()
bsu = [f'BSU_{i[3:]}' for i in bsu]
new_df = pd.DataFrame()
new_df['bsu'] = bsu
output_folder = '/Users/andreas/Bacillus/Bioinformatics/RNAseq_jag_khpa_kre/X204SC22071444-Z01-F001/Analysis/DEG_significant/'
new_df.to_csv(os.path.join(output_folder, 'kre_feedback_loop_sig_less_than0_genes.tsv'), sep = '\t', header = None, index = False)